# Plotting codes that need to be moved to Bokeh: Precision/Recall versus Redshift Plots

Code orginator: Renee Hlozek

In [ ]:
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl
from matplotlib import cm
from matplotlib import colors as mpcm

In [ ]:
import pickle
dir = '/Users/reneehlozek/Dropbox/pr_curves/'
redshifts = [0.0,  0.1723 ,0.3446, 0.5169, 0.6892, 0.8615, 1.0338 ,1.2061, 1.3784, 1.5507, 1.723 , 1.8953, 2.0676 ,2.2399, 2.4122] #, 2.5845, ] #2.7568, 2.9291 ,3.1014, 3.2737,  3.446 ]

names = ['SNIa','SN-Ibc','SN-II','SNIa-91bg',  'SNIax','KN','SLSN-I','Other', 
                   'TDE','AGN','RRlyrae', 'Mdwarf',                   'EBE', 'MIRA','$\mu$Lens-Single']

mids = [r + (redshifts[1]-redshifts[0])/2 for r in redshifts[0:-1]]
#precision_MT_0.0_0.1723.pkl
classifiers = ['MT', 'MS', 'KB']

import planckStyle as s
from matplotlib import rcParams, rc
g=s.plotter

g.settings.tight_layout=True
g.settings.lab_fontsize=30
g.settings.legend_frac_subplot_margin=-0.04

params = {'backend': 'pdf',
          'xtick.labelsize': 23,
          'ytick.major.pad': 8,
          'xtick.major.pad': 8,
          'ytick.labelsize': 23,
          'font.family':'sans-serif',
          'font.sans-serif':'FreeSans'}
rcParams.update(params)
rcParams['figure.figsize'] =8,6

colors=['r', 'm', 'b']
ntypes=14
precision_mat = np.zeros((ntypes,(len(redshifts)-1),3))
recall_mat = np.zeros((ntypes,(len(redshifts)-1),3))

for cz in range(len(redshifts)-1):
    low = redshifts[cz]
    high = redshifts[cz+1]

#    print((high-low)/2)
    for cc in range(len(classifiers)):
        file = dir+'/precisions_%s/'%classifiers[cc]+'precision_%s_%s_%s.pkl'%(classifiers[cc],str(low), str(high))

        recfile = dir+'/recalls_%s/'%classifiers[cc]+'recall_%s_%s_%s.pkl'%(classifiers[cc],str(low), str(high))
        threshfile = dir+'/thresholds_%s/'%classifiers[cc]+'threshold_%s_%s_%s.pkl'%(classifiers[cc],str(low), str(high))
        with open(file, 'rb') as pickle_file:
            precisions = pickle.load(pickle_file)
#            print(precisions)
        with open(threshfile, 'rb') as pickle_file:
            threshs = pickle.load(pickle_file)


        with open(recfile, 'rb') as pickle_file:
            recs = pickle.load(pickle_file)



        for ind in range(ntypes):
            try:
                indthreshs = np.where(threshs[ind+1] < 0.7)[0][-1]
            except:
                indthreshs = np.where(threshs[ind+1]==np.max(threshs[ind+1]))[0]

            threshval = threshs[ind+1][indthreshs]
            precision_mat[ind,cz,cc] = precisions[ind+1][indthreshs]
            recall_mat[ind,cz,cc] = recs[ind+1][indthreshs]

            print(np.mean(precisions[ind+1]), threshval, 'precision', 'threshval', 'classifier: ', classifiers[cc])

#        print(np.mean(precisions[1]), classifiers[cc], low+(high-low)/2.)
        #pl.plot(precision_mat[1], label='classifier %s'%classifiers[cc])

for i in range(ntypes):
    pl.clf()
    fig = pl.figure(figsize=(10,8))
    pl.plot(mids,precision_mat[i,:,0], color=colors[0], label='Major Tom' )
    pl.plot(mids,precision_mat[i,:,1], color=colors[1], label='Mike & Silogram')
    pl.plot(mids,precision_mat[i,:,2], color=colors[2],label='Kyle Boone')
    pl.plot(mids,recall_mat[i,:,0], color=colors[0], linestyle='--')
    pl.plot(mids,recall_mat[i,:,1], color=colors[1], linestyle='--')
    pl.plot(mids,recall_mat[i,:,2], color=colors[2], linestyle='--')

    pl.plot(mids, 10*(recall_mat[i,:,2]/recall_mat[i,:,2]), '-', color='k', label='precision')
    pl.plot(mids, 10*(recall_mat[i,:,2]/recall_mat[i,:,2]), '--', color='k', label='recall')


    leg =    pl.legend(loc='best', fontsize=20)
    leg.draw_frame(False)
    pl.xlabel('Redshift z', fontsize=25)
    pl.ylabel('Precision/Recall', fontsize=25)
    pl.axis([0,2.6, -0.05,1.1])
    pl.savefig(dir+'huh_pr_%s.png'%names[i])